<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Generative Adverserial Networks (GANs). In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a RNN classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection
* <a href="#p3">Part 3</a>: Describe the components of an autoencoder
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN/LSTM to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [1]:
from tensorflow.keras.datasets import reuters

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [2]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [39]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

batch_size = 46
max_features = len(word_index.values())
maxlen = 200

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)


print('Build model...')
model = Sequential()
model.add(Embedding(max_features+1, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(46, activation='softmax')) # softmax is appropriate for multiclass problems


8982 train sequences
2246 test sequences
Pad sequences (samples x time)
X_train shape: (8982, 200)
X_test shape: (2246, 200)
Build model...


In [41]:
y_train.max(), y_train.min() 
# range of values (0:45) -> a non-binary classification problem

# Get the baseline -> .3506
import pandas as pd
targDF = pd.DataFrame({'class': y_train})
targDF['class'].value_counts(normalize=True, dropna = False)


3     0.350590
4     0.214986
19    0.061456
1     0.049878
16    0.047651
11    0.041527
20    0.030728
13    0.018815
8     0.016477
10    0.014139
21    0.012469
9     0.011356
25    0.011022
2     0.008461
24    0.007793
18    0.007348
0     0.006123
36    0.005455
30    0.005344
12    0.005233
6     0.005233
34    0.005010
28    0.004676
23    0.004676
31    0.004565
17    0.004231
40    0.003785
32    0.003451
41    0.003340
26    0.002672
39    0.002561
43    0.002561
14    0.002561
15    0.002449
29    0.002227
5     0.002115
22    0.002115
38    0.002115
37    0.001893
45    0.001893
7     0.001781
42    0.001559
27    0.001559
35    0.001559
44    0.001336
33    0.001225
Name: class, dtype: float64

In [42]:
# You should only run this cell once your model has been properly configured

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc) #acc = .3664 w/sigmoid vs .5053 w/softmax 


Train...
Train on 8982 samples, validate on 2246 samples
8982/8982 [==============================] - 93s 10ms/sample - loss: 2.3701 - accuracy: 0.4036 - val_loss: 1.9574 - val_accuracy: 0.5053
2246/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

### Results of above RNN(LSTM): 
* Overall score: 1.9574
* Overall accuracy: 0.5053

## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

Here, padding is done (using the pad_sequences method) to ensure that raw input sequences are the same length. Deep learning models require that all sequences have the same length. Adding zeros to shorter sequences, so that they match the length of the longest sequence, is how padding circumvents this problem. 


## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

An LSTM enabled RNN is able to place a greater weight on recent (short-term) events while retaining, to a degree, older (long-term) information. General RNNs lack this sophistication. 


## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

One use case of LSTMs is for simple text-generation. Training an LSTM on source-texts can enable the generation of new text with a comparable flavor/tone. Another use of LSTMs is text-classification whereby text can be classified according to a training set of X(the text samples) and y(the class) data. Another very useful application of LSTMs is as a baby-name generator. By training an LSTM on a list of baby names, wholy unique names can be procured. 

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [46]:
!pip install google_images_download

     |████████████████████████████████| 911kB 744kB/s eta 0:00:01
  Stored in directory: /Users/Person/Library/Caches/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


In [79]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "lilly frog pond", "limit": 5, "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = lilly frog pond
Evaluating...
Starting Download...


Unfortunately all 5 could not be downloaded because some images were not downloadable. 0 is all we got for this search filter!

Errors: 0



At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goals* 
- Check for fish or other labels
- Create a matplotlib visualizations of the images and your prediction as the visualization label

In [78]:
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
model = ResNet50(weights='imagenet')

In [98]:
# You've got something to do in this cell. ;)

import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

# Define a function to output an img
def process_img_path(img_path):
    return image.load_img(img_path, target_size=(224, 224))

# Define a function to check if img contains frog
def img_contains_frog(img):
    """ Scans image for Frogs
    
    Should return a boolean (True/False) if a frog is in the image.
    
    Inputs:
    ---------
    img:  Precrossed image ready for prediction. The `process_img_path`             function should already be applied to the image. 
    
    Returns: 
    ---------
    frogs (boolean):  TRUE or FALSE - There are frogs in the image.
    
    """
    test_img = image.img_to_array(img)
    test_img = np.expand_dims(test_img, axis=0)
    result = model.predict(test_img)
    if result[0][0] >= .75:
        prediction = "1)Frog"
    else:
        prediction = "0)Toad"
    print(prediction)
    return prediction

In [100]:
# Test out the functions defined above:
img_contains_frog(process_img_path("./downloads/lilly frog pond/2.Nature-Bull-Frog-Pond-Green-Lily-Pad-Frog-2526024.jpg"))

0)Toad


'0)Toad'

In [108]:
# Show the image:

# Prep for reception
!pip install Pillow
from PIL import Image

# Pick an image
image = Image.open('./downloads/lilly frog pond/2.Nature-Bull-Frog-Pond-Green-Lily-Pad-Frog-2526024.jpg')
image.show(); 
# As can be seen, the model may be a bit off 


#### Stretch Goal: Displaying Predictions

In [ ]:
import matplotlib.pyplot as plt



<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

__*Your Answer: An autoencoder could be used in generative modeling; learning how to classify handwritten, alphanumeric characters. *__ 

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist? 
    My strongest area in Data Science is in 
- What area of Data Science would you most like to learn more about, and why?
    At the moment, the practicle application of sentiment analysis (NLP) is something I am trying to learn more about. 
- Where do you think Data Science will be in 5 years?
    I think in 5 years' time Data Science can grow to be taught, at a basic level, in many fields. I don't see why the subject material cannot be facilitated for more widespread use and direct adoption by managers. The barriers to entry, I expect, will decrease.  
- What are the threats posed by AI to our society? Job loss and an accelerated decline in the health of the Biosphere are notable concerns associated with the rise of an artificial general intelligence.  
- How do you think we can counteract those threats? By introducing humanitarian ideals into economics one might hope that a new global economic system not wholly concerned with short-term monetary profit might gain acceptance.  
- Do you think achieving General Artifical Intelligence is ever possible? I think that there is a greater statistical chance of an AGI being produced than there is for an AGI never being produced.  

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [45]:
#from IPython.display import HTML

#HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")